# Imports

In [96]:
import pandas             as pd
import plotly             as plt
import numpy              as np
import seaborn            as sns
import inflection
import math


from IPython.core.display import HTML

## Helper Functions

In [97]:
def personal_settings():
    
#     %matplotlib inline
#     %pylab inline
    
#      plt.style.use( 'bmh' )
#    plt.rcParams['figure.figsize'] = [20, 10]
#     plt.rcParams['font.size'] = 24
    
    display(HTML('<style>.container{width:90% !important;}</style>'))
#     pd.options.display.max_columns = None
#     pd.options.display.max_rows = None
#     pd.set_option( 'display.expand_frame_repr', False )
    
    #sns.set()
    sns.set(rc={'figure.figsize':(15,10)})
    sns.set_theme(style = 'darkgrid', font_scale = 1.5)

In [76]:
personal_settings();

## Loading Data

In [77]:
df_sales_raw = pd.read_csv('../data/raw/train.csv', low_memory=False)
df_store_raw = pd.read_csv('../data/raw/store.csv', low_memory=False)

# merging datsets
df_raw = pd.merge(df_sales_raw, df_store_raw, how='left', on='Store')

# DATA DESCRIPTION

In [78]:
df1=df_raw.copy()

## Rename Columns

In [79]:
df1.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [80]:
cols_old=['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
           'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
           'CompetitionDistance', 'CompetitionOpenSinceMonth',
           'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
           'Promo2SinceYear', 'PromoInterval']


# changing the name of the columns
snakecase = lambda x: inflection.underscore(x)
cols_new = list(map(snakecase, cols_old))

# renaming dataset columns
df1.columns = cols_new
print(df1.columns)

Index(['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
       'state_holiday', 'school_holiday', 'store_type', 'assortment',
       'competition_distance', 'competition_open_since_month',
       'competition_open_since_year', 'promo2', 'promo2_since_week',
       'promo2_since_year', 'promo_interval'],
      dtype='object')


## Data Dimension

In [81]:
print('Number of rows: {}' .format(df1.shape[0]))
print('Number of columns: {}' .format(df1.shape[1]))

Number of rows: 1017209
Number of columns: 18


## Data Types

In [82]:
print(df1.dtypes)
df1.head()

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object


,store,day_of_week,date,sales,customers,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance,competition_open_since_month,competition_open_since_year,promo2,promo2_since_week,promo2_since_year,promo_interval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


## Check NA 

In [83]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

## Fillout NA

In [84]:
# ======= competition_distance

#checking maximum value
df1['competition_distance'].max() # The maximum distance is:  75860.0

# replacing each NA
df1['competition_distance'] = df1['competition_distance'].apply(lambda x:2000000.0 if math.isnan(x) else (x))

In [85]:
df1['date'] = pd.to_datetime(df1['date'])

#  ======= competition_open_since_month
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

#  ======= competition_open_since_year
df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)


In [86]:
# ======= promo2_since_week
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)

# ======= promo2_since_year
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)


In [87]:
# ======= promo_interval

month_map = { 1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
              7: 'Jul', 8: 'Ago', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

# filling Na with "0"
df1['promo_interval'].fillna(0, inplace=True)

# finding the month name given month number by mapping the "month_map" dictionary
df1 ['month_map'] = df1['date'].dt.month.map(month_map)


df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: 0 if x['promo_interval'] == 0
                                                                  else 1 if x['month_map'] in x['promo_interval'].split(',')
                                                                  else 0, axis=1 )

In [88]:
df1[['competition_distance','competition_open_since_month','competition_open_since_year',
     'promo2_since_week','promo2_since_year','promo_interval', 'month_map', 'is_promo']].sample(5)

,competition_distance,competition_open_since_month,competition_open_since_year,promo2_since_week,promo2_since_year,promo_interval,month_map,is_promo
813097,1860.0,9.0,2012.0,40.0,2012.0,"Jan,Apr,Jul,Oct",Jul,1
974114,1600.0,4.0,2009.0,6.0,2013.0,0,Feb,0
797119,170.0,7.0,2013.0,35.0,2012.0,"Mar,Jun,Sept,Dec",Jul,0
592966,9230.0,4.0,2004.0,37.0,2009.0,"Jan,Apr,Jul,Oct",Jan,1
661903,6630.0,11.0,2013.0,46.0,2013.0,0,Nov,0


## Change Variable Types

In [89]:
print(df1.dtypes)
df1.sample()

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
month_map                               object
is_promo                                 int64
dtype: object


,store,day_of_week,date,sales,customers,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance,competition_open_since_month,competition_open_since_year,promo2,promo2_since_week,promo2_since_year,promo_interval,month_map,is_promo
377826,319,6,2014-08-02,5273,576,1,0,0,0,a,c,570.0,9.0,2012.0,1,27.0,2012.0,"Mar,Jun,Sept,Dec",Ago,0


In [90]:
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype(np.int64)
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype(np.int64)

df1['promo2_since_week'] = df1['promo2_since_week'].astype(np.int64)
df1['promo2_since_year'] = df1['promo2_since_year'].astype(np.int64)

## Descriptive Statistical

In [91]:
num_df1 = df1.select_dtypes(include=['int64','float64'])
cat_df1 = df1.select_dtypes(exclude=['int64','float64','datetime64[ns]'])

### Numerical Attributes

In [92]:
# central tendendy - mean, median
ct1 = pd.DataFrame(num_df1.apply(np.mean)).T
ct2 = pd.DataFrame(num_df1.apply(np.median)).T

# dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_df1.apply(np.std)).T
d2 = pd.DataFrame(num_df1.apply(min)).T
d3 = pd.DataFrame(num_df1.apply(max)).T
d4 = pd.DataFrame(num_df1.apply(lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_df1.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_df1.apply(lambda x: x.kurtosis)).T

# concatenate 
m = pd.concat([d2,d3,d4,ct1,ct2,d1,d5,d6]).T.reset_index()
m.columns = ['attributes','min','max','range','mean','median','std','skew','kurtosis']
m

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,store,1.0,1115.0,1114.0,558.429727,558.0,321.908493,-0.000955,<bound method NDFrame._add_numeric_operations....
1,day_of_week,1.0,7.0,6.0,3.998341,4.0,1.99739,0.001593,<bound method NDFrame._add_numeric_operations....
2,sales,0.0,41551.0,41551.0,5773.818972,5744.0,3849.924283,0.64146,<bound method NDFrame._add_numeric_operations....
3,customers,0.0,7388.0,7388.0,633.145946,609.0,464.411506,1.59865,<bound method NDFrame._add_numeric_operations....
4,open,0.0,1.0,1.0,0.830107,1.0,0.375539,-1.758045,<bound method NDFrame._add_numeric_operations....
5,promo,0.0,1.0,1.0,0.381515,0.0,0.485758,0.487838,<bound method NDFrame._add_numeric_operations....
6,school_holiday,0.0,1.0,1.0,0.178647,0.0,0.383056,1.677842,<bound method NDFrame._add_numeric_operations....
7,competition_distance,20.0,2000000.0,1999980.0,10610.588099,2330.0,101810.6491,19.378002,<bound method NDFrame._add_numeric_operations....
8,competition_open_since_month,1.0,12.0,11.0,6.786849,7.0,3.311085,-0.042076,<bound method NDFrame._add_numeric_operations....
9,competition_open_since_year,1900.0,2015.0,115.0,2010.32484,2012.0,5.515591,-7.235657,<bound method NDFrame._add_numeric_operations....


### Categorical Attributes

In [93]:
cat_df1.apply(lambda x: x.unique().shape[0])

state_holiday      4
store_type         4
assortment         3
promo_interval     4
month_map         12
dtype: int64

In [99]:
# filtering sales that happened during a state holiday
aux1 = df1[(df1['state_holiday']!='0') & (df1['sales']>0)]

plt.subplot(1,3,1)
sns.boxplot(x='state_holiday', y='sales', data=aux1);

plt.subplot(1,3,2)
sns.boxplot(x='store_type', y='sales', data=aux1);

plt.subplot(1,3,3)
sns.boxplot(x='assortment', y='sales', data=aux1);

AttributeError: module 'plotly' has no attribute 'subplot'